In [16]:
from PIL import Image
import numpy as np
from collections import deque
import os

def flood_fill_transparent(img_array, threshold=230):
    height, width, _ = img_array.shape
    mask = np.all(img_array[:, :, :3] > threshold, axis=2)
    
    queue = deque([(0, 0)])
    while queue:
        y, x = queue.popleft()
        if not mask[y, x]:
            continue
        
        mask[y, x] = False
        img_array[y, x, 3] = 0
        
        for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            ny, nx = y + dy, x + dx
            if 0 <= ny < height and 0 <= nx < width and mask[ny, nx]:
                queue.append((ny, nx))
    
    return img_array

def smooth_edges(img_array, smoothing_factor=0.3):
    alpha = img_array[:, :, 3]
    edges = np.zeros_like(alpha, dtype=bool)
    
    edges[1:] |= (alpha[1:] == 0) != (alpha[:-1] == 0)
    edges[:-1] |= (alpha[:-1] == 0) != (alpha[1:] == 0)
    edges[:, 1:] |= (alpha[:, 1:] == 0) != (alpha[:, :-1] == 0)
    edges[:, :-1] |= (alpha[:, :-1] == 0) != (alpha[:, 1:] == 0)
    
    for i in range(-1, 2):
        for j in range(-1, 2):
            if i == 0 and j == 0:
                continue
            shifted = np.roll(np.roll(edges, i, axis=0), j, axis=1)
            img_array[:, :, 3] = np.where(
                shifted & (alpha > 0),
                img_array[:, :, 3] * (1 - smoothing_factor),
                img_array[:, :, 3]
            )
    
    return img_array

def process_gif(input_path, output_path, threshold=230, smoothing_factor=0.3):
    with Image.open(input_path) as img:
        duration = img.info.get('duration', 1200)
        loop = img.info.get('loop', 0)
        
        frames = []
        for frame in iter(lambda: img.copy().convert("RGBA"), None):
            img_array = np.array(frame)
            
            img_array = flood_fill_transparent(img_array, threshold)
            # img_array = smooth_edges(img_array, smoothing_factor)
            
            frames.append(Image.fromarray(img_array))
            
            try:
                img.seek(img.tell() + 1)
            except EOFError:
                break
    
    if len(frames) > 1:
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            optimize=True,
            duration=duration,
            loop=loop,
            disposal=2,
            transparency=0
        )
    else:
        frames[0].save(output_path, format="PNG")


In [19]:
process_gif(r"C:\Users\DELL\Desktop\skies\assets\cloudy.gif", r"C:\Users\DELL\Desktop\skies\assets\cloudy_day.gif",200)